In [1]:
"""
 * @ Author: Lucas Glasner (lgvivanco96@gmail.com)
 * @ Create Time: 1969-12-31 21:00:00
 * @ Modified by: Lucas Glasner, 
 * @ Modified time: 2024-03-21 15:35:44
 * @ Description:
 * @ Dependencies:
 */
"""

import os

from grass_session import Session
import grass.script as gs
from grass.pygrass.modules.grid import GridModule

import geopandas as gpd
import rioxarray as rxr
import numpy as np
import pandas as pd
from geocube.vector import vectorize

NCORES = 8
wd = '/home/lucas/R-storage/P-projects/geobasins'
os.chdir(wd)

In [49]:

if os.path.isdir('data/GRASS'):
    session = Session()
    session.open(gisdb=f'{wd}/data', location='GRASS')
else:
    PERMANENT = Session()
    PERMANENT.open(gisdb=f'{wd}/data', location='GRASS', create_opts='EPSG:4326')
    PERMANENT.close()

    session = Session()
    session.open(gisdb=f'{wd}/data', location='GRASS', mapset='geobasins', create_opts='')
    session
    

In [50]:
def space(char='%',num=79):
    print(char*num)
    return

def grass_loadraster(path, 
                     output_name,
                     kwargs_rimport = dict(flags='o', overwrite=True),
                     kwargs_gregion = dict(flags='p')):
    space()
    print('Loading raster data from: ', path, '\n\n')
    gs.run_command("r.import",
                   input=path, output=output_name,
                   **kwargs_rimport)
    print('\n\nResults:')
    space()
    gs.run_command("g.region", raster=output_name, 
                   **kwargs_gregion)
    print('\n\n')
    return

def grass_processDEM(input_name,
                     kwargs_rhydrodem  = dict(overwrite = True,
                                              processes = 4,
                                              overlap   = 500),
                     kwargs_rwatershed = dict(overwrite = True,
                                              threshold = 500),
                     ):
    space()
    print('Sink removal using Lindsay et al. (2005)...\n\n')
    grid = GridModule("r.hydrodem",
                    input=input_name, 
                    output=f"{input_name}_nosink",
                    **kwargs_rhydrodem)
    grid.run()
    space()
    print('Calculate accumulation and drainage direction map using MFD...\n\n')
    gs.run_command("r.watershed", 
               elevation=f"{input_name}_nosink", 
               drainage=f"{input_name}_fdir", 
               accumulation=f"{input_name}_facc",
               **kwargs_rwatershed)
    print('\n\n')
    return grid
    

In [51]:
dataset_name = 'NASADEM'
grass_loadraster('data/NASADEM/NASADEM.tif', output_name=dataset_name)
grass_processDEM(input_name=dataset_name)


# Store output rasters to .tif files
gs.run_command('r.out.gdal', input=f"{dataset_name}_nosink", output='data/NASADEM/NASADEM_nosink.tif', format='GTiff', overwrite=True)
gs.run_command('r.out.gdal', input=f"{dataset_name}_facc", output='data/NASADEM/NASADEM_facc.tif', format='GTiff', overwrite=True)
gs.run_command('r.out.gdal', input=f"{dataset_name}_fdir", output='data/NASADEM/NASADEM_fdir.tif', format='GTiff', overwrite=True)

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
Loading raster data from:  data/NASADEM/NASADEM.tif 




Over-riding projection check
Importing raster map <NASADEM>...
   0   3   6   9  12  15  18  21  24  27  30  33  36  39  42  45  48  51  54  57  60  63  66  69  72  75  78  81  84  87  90  93  96  99 100




Results:
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
projection: 3 (Latitude-Longitude)
zone:       0
datum:      wgs84
ellipsoid:  wgs84
north:      31:59:59.5S
south:      32:59:59.5S
west:       71:00:00.5W
east:       70:00:00.5W
nsres:      0:00:01
ewres:      0:00:01
rows:       3600
cols:       3600
cells:      12960000



%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
Sink removal using Lindsay et al. (2005)...




Load elevation map
   0Load elevation map
   0Load elevation map
   0Load elevation map
   0   3   3   3   6   3   6   9   6   6   9  12   9  12  15   9  12  15  18  12  18  21  15  15  21  24  18  27  24  18  21  30  27  21  33  30  24  24  36  33  27  39  27  36  30  42  39  30  45  33  42  33  48  45  36  51  36  48  54  39  51  39  57  42  54  60  42  57  45  63  45  60  66  48  63  69  48  51  66  72  51  75  54  69  54  78  72  57  81  57  75  60  84  60  78  87  63  81  63  90  66  84  93  66  87  69  96  69  90  99  72 100
  93  72Remove one cell extremas
   0   3   6   9  75  12  96  15  18  75  21  24  27  78  99  30 100
  33  78  36  39Remove one cell extremas
   0  42   3  45   6  81  48   9  51  12  54  81  57  15  60  18  63  21  84  66  24  69  27  84  72  30  75  33  78  36  87  81  39  84  42  87  45  87  90  48  93  51  96  54  90  99 100
Set edge points
   0   3   6  57   9  12  15  18  21  24  27  30  33  36  60  39  42  45  48  51  54  57  60  63  90  66  63  69  7

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
Calculate accumulation and drainage direction map using MFD...




SECTION 1a (of 5): Initiating Memory.
SECTION 1b (of 5): Determining Offmap Flow.
   0   4   8  12  16  20  24  28  32  36  40  44  48  52  56  60  64  68  72  76  80  84  88  92  96 100
SECTION 2: A* Search.
   0   2   4   6   8  10  12  14  16  18  20  22  24  26  28  30  32  34  36  38  40  42  44  46  48  50  52  54  56  58  60  62  64  66  68  70  72  74  76  78  80  82  84  86  88  90  92  94  96  98 100
SECTION 3a: Accumulating Surface Flow with MFD.
   1   3   5   7   9  11  13  15  17  19  21  23  25  27  29  31  33  35  37  39  41  43  45  47  49  51  53  55  57  59  61  63  65  67  69  71  73  75  77  79  81  83  85  87  89  91  93  95  97  99 100
SECTION 3b: Adjusting drainage directions.
   1   3   5   7   9  11  13  15  17  19  21  23  25  27  29  31  33  35  37  39  41  43  45  47  49  51  53  55  57  59  61  63  65  67  69  71  73  75  77  79  81  83  85  87  89  91  93  95  97  99 100
SECTION 4: Watershed determination.
   0   2   4   6   8  10  12  14  16  18  20  22 

Checking GDAL data type and nodata value...
   2   5   8  11  14  17  20  23  26  29  32  35  38  41  44  47  50  53  56  59  62  65  68  71  74  77  80  83  86  89  92  95  98 100
Using GDAL data type <UInt16>
Exporting raster data to GTiff format...
   2   5   8  11  14  17  20  23  26  29  32  35  38  41  44  47  50  53  56  59  62  65  68  71  74  77  80  83  86  89  92  95  98 100
r.out.gdal complete. File <data/NASADEM/NASADEM_nosink.tif> created.
Checking GDAL data type and nodata value...
   2   5   8  11  14  17  20  23  26  29  32  35  38  41  44  47  50  53  56  59  62  65  68  71  74  77  80  83  86  89  92  95  98 100
Using GDAL data type <Float64>
Exporting raster data to GTiff format...
ERROR 6: data/NASADEM/NASADEM_facc.tif, band 1: SetColorTable() only supported for Byte or UInt16 bands in TIFF format.
   2   5   8  11  14  17  20  23  26  29  32  35  38  41  44  47  50  53  56  59  62  65  68  71  74  77  80  83  86  89  92  95  98 100
r.out.gdal complete. File <data/

In [44]:
# # Sink removal using Lindsay et al. (2005) [MULTICORE]
# grid = GridModule("r.hydrodem",
#                   input="dem", 
#                   output = "dem_nosink",
#                   overwrite = True,
#                   processes=NCORES,
#                   overlap=500)
# grid.run()

In [11]:
# # calculate accumulation raster map and drainage direction raster map using MFD: multiple flow direction
# gs.run_command("r.watershed", 
#                elevation="elevation_filled", 
#                threshold=500,
#                drainage= "fdir", 
#                accumulation="facc",
#                overwrite = True)


SECTION 1a (of 5): Initiating Memory.
SECTION 1b (of 5): Determining Offmap Flow.
   0   4   8  12  16  20  24  28  32  36  40  44  48  52  56  60  64  68  72  76  80  84  88  92  96 100
SECTION 2: A* Search.
   0   2   4   6   8  10  12  14  16  18  20  22  24  26  28  30  32  34  36  38  40  42  44  46  48  50  52  54  56  58  60  62  64  66  68  70  72  74  76  78  80  82  84  86  88  90  92  94  96  98 100
SECTION 3a: Accumulating Surface Flow with MFD.
   1   3   5   7   9  11  13  15  17  19  21  23  25  27  29  31  33  35  37  39  41  43  45  47  49  51  53  55  57  59  61  63  65  67  69  71  73  75  77  79  81  83  85  87  89  91  93  95  97  99 100
SECTION 3b: Adjusting drainage directions.
   1   3   5   7   9  11  13  15  17  19  21  23  25  27  29  31  33  35  37  39  41  43  45  47  49  51  53  55  57  59  61  63  65  67  69  71  73  75  77  79  81  83  85  87  89  91  93  95  97  99 100
SECTION 4: Watershed determination.
   0   2   4   6   8  10  12  14  16  18  20  22 

In [62]:
basinout     = gpd.read_file('data/watershed_outlets.shp')
grid         = rxr.open_rasterio('data/NASADEM/NASADEM.tif', masked=True).squeeze()

snap_coords  = [grid.sel(x=x, y=y, method='nearest') 
                for x,y in zip(basinout.geometry.x,
                               basinout.geometry.y)]
snap_coords       = [(snap.x.item(),snap.y.item()) for snap in snap_coords]
snap_coords       = np.array(snap_coords)

basinout_snap = gpd.points_from_xy(x=snap_coords[:,0], y=snap_coords[:,1])
basinout_snap = gpd.GeoDataFrame(geometry=basinout_snap)
basinout_snap['id'] = basinout['id']
basinout_snap.index = basinout_snap['id'].values
basinout_snap.to_file('data/NASADEM/basinout_snappoint.shp')

In [63]:
# Watershed outlets
gs.run_command("v.import",  input="data/NASADEM/basinout_snappoint.shp", output="basinout_snap", flags = "o", overwrite=True)

No projection information available for layer <basinout_snappoint>
Over-riding projection check
Check if OGR layer <basinout_snappoint> contains polygons...
   0  25  50  75 100
Creating attribute table for layer <basinout_snappoint>...
Importing 4 features (OGR layer <basinout_snappoint>)...
   0  25  50  75 100
-----------------------------------------------------
Building topology for vector map <basinout_snap@geobasins>...
Registering primitives...
Input <data/NASADEM/basinout_snappoint.shp> successfully imported without
reprojection


In [80]:
for bid in basinout_snap.index:
    geoms  = basinout_snap.geometry
    coords = geoms.x.loc[bid],geoms.y.loc[bid]
    gs.run_command("r.water.outlet", input='fdir', output=f'{bid}_Basin',
                   coordinates=coords, overwrite=True)
    gs.run_command('r.out.gdal', input=f'{bid}_Basin',
                   output=f'data/NASADEM/{bid}_Basin.tif',
                   format='GTiff', overwrite=True)
    
    # Vectorize and delete basin bool raster
    basin = rxr.open_rasterio(f'data/NASADEM/{bid}_Basin.tif', masked=True)
    basin.name = 'gauge_id'
    basin = vectorize(basin)
    basin['gauge_id'] = bid
    basin = basin.set_crs(4326)
    basin = basin.dissolve(by='gauge_id')
    basin = basin.reset_index()
    basin.to_file(f'data/NASADEM/{bid}_Basin.shp')
    os.remove(f'data/NASADEM/{bid}_Basin.tif')


    
    
#     coords = [q_grass_snap.x[basin], q_grass_snap.y[basin]]
#     gs.run_command("r.water.outlet", input="fdir", output = "basin_i", coordinates = coords, overwrite=True)
#     gs.run_command('r.out.gdal', input="basin_i", output=  'GIS South/Basins/Basin_{}.tif'.format(basin), format='GTiff', overwrite=True)
#     data = rioxr.open_rasterio('GIS South/Basins/Basin_{}.tif'.format(basin))
#     data.name = "gauge_id"
#     data = vectorize(data)
#     data["gauge_id"] = basin
#     data = data.set_crs(4326)
#     data = data.dissolve(by='gauge_id')
#     data = data.reset_index()
#     df.append(data)

# df = pd.concat(df)
# df = df.reset_index()

# df["gauge_name"] = q_location.gauge_name
# df["institutio"] = q_location.institution
# df["int_area"]   = df.to_crs(32719).area / 1e6

# df.to_file("GIS South/Basins_PMETobs.shp")

   0   6  12  18  24  30  36  42  48  54  60  66  72  78  84  90  96 100
Checking GDAL data type and nodata value...
   2   5   8  11  14  17  20  23  26  29  32  35  38  41  44  47  50  53  56  59  62  65  68  71  74  77  80  83  86  89  92  95  98 100
Using GDAL data type <Byte>
Input raster map contains cells with NULL-value (no-data). The value 255
will be used to represent no-data values in the input map. You can specify
a nodata value with the nodata option.
Exporting raster data to GTiff format...
   2   5   8  11  14  17  20  23  26  29  32  35  38  41  44  47  50  53  56  59  62  65  68  71  74  77  80  83  86  89  92  95  98 100
r.out.gdal complete. File <data/NASADEM/1_Basin.tif> created.
   0   6  12  18  24  30  36  42  48  54  60  66  72  78  84  90  96 100
Checking GDAL data type and nodata value...
   2   5   8  11  14  17  20  23  26  29  32  35  38  41  44  47  50  53  56  59  62  65  68  71  74  77  80  83  86  89  92  95  98 100
Using GDAL data type <Byte>
Input ras

In [13]:
# # Performs stream network extraction [MULTICORE]
# grid = GridModule("r.stream.extract",
#                   elevation="elevation_filled",  
#                   threshold=10000,
#                   accumulation= "facc", 
#                   stream_vector="stream_v", 
#                   stream_raster="stream_r",
#                   overwrite = True,
#                   processes=NCORES,
#                   overlap=500)
# grid.run()

In [81]:
session.close()